In [28]:
#we can use the slope predict the output of our input
#For a 3d linear regression -> we would need two inputs else if it was a 2d linear regression then
#we just need 1 input 

!pip install -q sklearn
%tensorflow_version 2.x

import tensorflow as tf

from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd #data analytics tool that allows you to manipulate data
import numpy as np #multi dimensional calculations -> modified arrays

import matplotlib.pyplot as plt #visualize the data by graphs
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc



#we did not have to import train_test_split since already have two data sets
dfTrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfTest = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
yTrain = dfTrain.pop('survived')
yTest = dfTest.pop('survived')


#dfTrain will no longer have the survived option

categorical = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']

numerical = ['age', 'fare']

feature_columns = []

for i in categorical: 
  vocab = dfTrain[i].unique()

  #feature_column
  #categorical_column_with_vocabulary


  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(i, vocab))

  #feature_column
  #numeric_column
for j in numerical:
  feature_columns.append(tf.feature_column.numeric_column(j, dtype=tf.float32))


feature_columns



#TRAINING

#loading batches with 32 info
#epoch the number of ways you can show a certain data to the model(having to input it basically)
def make_input_fn(data_df, label_df, num_epochs=40, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use


#putting in the data
train_input_fn = make_input_fn(dfTrain, yTrain)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfTest, yTest, num_epochs=1, shuffle=False)

#creating the linear regression line
#linear model(I think)
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns)

#training the model by passing in the input
linear_est.train(train_input_fn) #train


#Testing the model by pasing in the test data
res = linear_est.evaluate(eval_input_fn)

clear_output() #clears out the loading phase of the algo

res['accuracy']

linear_est


In [30]:
predictions = list(linear_est.predict(eval_input_fn))

print(dfTest.loc[10])
print(predictions[10]['probabilities'][1])

print(yTest[10])



INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/canned/linear.py:1478: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  getter=tf.compat.v1.get_variable)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpt1urip89/model.ckpt-800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                       female
age                           28
n_siblings_spouses             0
parch                          0
fare                        7.75
class                      Third
deck                     unknown
embark_town           Queenstown
alone                          y
Name: 10, dtype: object
0.6722688
1
